# Phase 4: Model Tuning & Explainability — Predicting 30-Day Hospital Readmission

**Goal:**  
Improve baseline model performance from Phase 3 through hyperparameter tuning, probability calibration, and model explainability.

**Key steps in this notebook:**
1. **Load cleaned dataset** from Phase 2.
2. **Reuse preprocessing pipeline** from Phase 3 (encoding + scaling).
3. **Hyperparameter tuning**:
   - Logistic Regression (`GridSearchCV`, scoring by Precision-Recall AUC)
   - Random Forest (`GridSearchCV`, scoring by Precision-Recall AUC)
4. **Evaluate tuned models**:
   - ROC AUC
   - Precision-Recall AUC
   - Best decision threshold (F1-optimized)
5. **Probability calibration** (isotonic) for trustworthy risk scores.
6. **Explainability with SHAP**:
   - Global feature importance
   - Individual prediction explanations
7. **Save final model and configuration** for deployment.

**Why this matters:**  
Tuned and interpretable models are essential for healthcare use cases.  
They ensure better recall of high-risk patients while giving clinicians insights into *why* a model predicts a readmission, increasing trust and adoption.


In [3]:
QUICK_MODE = False  

import numpy as np
import pandas as pd
from pathlib import Path
from tempfile import mkdtemp
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    roc_auc_score, average_precision_score, precision_recall_curve,
    f1_score, classification_report, confusion_matrix
)
from sklearn.calibration import CalibratedClassifierCV
import joblib, json

# Load cleaned data
df_clean = pd.read_csv(Path("../data/diabetic_data_clean.csv"))
TARGET = "readmit_30"

drop_cols = [c for c in ["encounter_id", "patient_nbr", "readmitted", TARGET] if c in df_clean.columns]
X = df_clean.drop(columns=drop_cols)
y = df_clean[TARGET]

print("Dataset:", X.shape, "| Positive rate:", round(float(y.mean()), 4))

# Preprocessing (no downsampling, keep rare categories)
cat_cols = X.select_dtypes(include="object").columns.tolist()
num_cols = X.select_dtypes(exclude="object").columns.tolist()

ohe = OneHotEncoder(
    handle_unknown="ignore",
    sparse_output=True,   # keep memory light
    # min_frequency=None  # keep full cardinality
)

preprocess = ColumnTransformer(
    transformers=[
        ("cat", ohe, cat_cols),
        ("num", StandardScaler(with_mean=False), num_cols),
    ],
    remainder="drop",
    sparse_threshold=1.0
)

# Train/test split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Train/Test:", X_tr.shape, X_te.shape)

# Cache preprocessing so OHE/scale isn't refit each fold/param
cache_dir = mkdtemp()


# Logistic Regression (full grid)
pipe_lr = Pipeline(
    steps=[
        ("prep", preprocess),
        ("clf", LogisticRegression(max_iter=2000, class_weight="balanced"))
    ],
    memory=cache_dir
)

grid_lr = {
    "clf__C": [0.25, 0.5, 1.0, 2.0, 4.0],
    "clf__penalty": ["l2"],
    "clf__solver": ["lbfgs", "liblinear"]
}

# Parallelize the grid; model itself is single-threaded
gs_lr = GridSearchCV(
    pipe_lr, grid_lr, scoring="average_precision", cv=3,
    n_jobs=4, pre_dispatch="2*n_jobs", verbose=2
)
gs_lr.fit(X_tr, y_tr)

print("Best PR AUC (LR):", round(float(gs_lr.best_score_), 4), "| Params:", gs_lr.best_params_)


# Random Forest (moderately large grid, stable on 4 cores)
pipe_rf = Pipeline(
    steps=[
        ("prep", preprocess),
        ("clf", RandomForestClassifier(
            class_weight="balanced_subsample",
            random_state=42,
            n_jobs=1   
        ))
    ],
    memory=cache_dir
)

# ~36 combos * 3 folds = 108 fits 
grid_rf = {
    "clf__n_estimators": [400, 800],
    "clf__max_depth": [None, 16, 28],
    "clf__min_samples_leaf": [1, 3, 5],
    "clf__max_features": ["sqrt", 0.4],
}

# For heavier search
# grid_rf = {
#     "clf__n_estimators": [400, 800, 1200],
#     "clf__max_depth": [None, 16, 28, 36],
#     "clf__min_samples_leaf": [1, 3, 5],
#     "clf__max_features": ["sqrt", 0.3, 0.5],
# }  # 3*4*3*3 = 108 combos (x3 folds = 324 fits) — longer

gs_rf = GridSearchCV(
    pipe_rf, grid_rf, scoring="average_precision", cv=3,
    n_jobs=4, pre_dispatch="2*n_jobs", verbose=2
)
gs_rf.fit(X_tr, y_tr)

print("Best PR AUC (RF):", round(float(gs_rf.best_score_), 4), "| Params:", gs_rf.best_params_)


# Evaluate tuned models + choose threshold
best_lr = gs_lr.best_estimator_
best_rf = gs_rf.best_estimator_

proba_lr = best_lr.predict_proba(X_te)[:, 1]
proba_rf = best_rf.predict_proba(X_te)[:, 1]

def eval_probs(name, y_true, proba):
    roc = roc_auc_score(y_true, proba)
    ap = average_precision_score(y_true, proba)
    prec, rec, thr = precision_recall_curve(y_true, proba)
    f1s = [f1_score(y_true, (proba >= t).astype(int)) for t in thr]
    t_best = thr[int(np.argmax(f1s))]
    yhat = (proba >= t_best).astype(int)
    print(f"\n{name} | ROC AUC: {round(float(roc),4)} | PR AUC: {round(float(ap),4)} | Best-F1 thr: {round(float(t_best),4)}")
    print(classification_report(y_true, yhat, digits=3))
    print("Confusion matrix @ best-F1:\n", confusion_matrix(y_true, yhat))
    return ap, t_best

ap_lr, thr_lr = eval_probs("LogReg (tuned)", y_te, proba_lr)
ap_rf, thr_rf = eval_probs("RandForest (tuned)", y_te, proba_rf)


# Pick winner & calibrate (isotonic)
if ap_rf >= ap_lr:
    winner_name, winner_est = "RF", best_rf
else:
    winner_name, winner_est = "LR", best_lr

cal = CalibratedClassifierCV(winner_est, method="isotonic", cv=3)
cal.fit(X_tr, y_tr)
proba_cal = cal.predict_proba(X_te)[:, 1]

roc_cal = roc_auc_score(y_te, proba_cal)
ap_cal = average_precision_score(y_te, proba_cal)
prec, rec, thr = precision_recall_curve(y_te, proba_cal)
f1s = [f1_score(y_te, (proba_cal >= t).astype(int)) for t in thr]
thr_cal = thr[int(np.argmax(f1s))]

print(f"\nCalibrated {winner_name} | ROC AUC: {round(float(roc_cal),4)} | PR AUC: {round(float(ap_cal),4)}")
print("Chosen threshold (best F1, calibrated):", round(float(thr_cal),4))


# Save model & deployment config
artifacts = Path("artifacts"); artifacts.mkdir(exist_ok=True)
final_model_name = f"final_calibrated_{winner_name.lower()}.joblib"
joblib.dump(cal, artifacts / final_model_name)

with open(artifacts / "deployment_config.json", "w") as f:
    json.dump({
        "model": final_model_name,
        "threshold": float(thr_cal),
        "metric_pr_auc": round(float(ap_cal), 4),
        "metric_roc_auc": round(float(roc_cal), 4),
        "quick_mode": False
    }, f, indent=2)

print("\nSaved:", final_model_name, "and deployment_config.json")


Dataset: (101766, 44) | Positive rate: 0.1116
Train/Test: (81412, 44) (20354, 44)
Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END ....clf__C=0.25, clf__penalty=l2, clf__solver=lbfgs; total time=  11.7s
[CV] END ....clf__C=0.25, clf__penalty=l2, clf__solver=lbfgs; total time=  12.4s
[CV] END ....clf__C=0.25, clf__penalty=l2, clf__solver=lbfgs; total time=  12.5s
[CV] END clf__C=0.25, clf__penalty=l2, clf__solver=liblinear; total time=  12.8s
[CV] END clf__C=0.25, clf__penalty=l2, clf__solver=liblinear; total time=  15.0s
[CV] END .....clf__C=0.5, clf__penalty=l2, clf__solver=lbfgs; total time=  14.1s
[CV] END clf__C=0.25, clf__penalty=l2, clf__solver=liblinear; total time=  15.1s
[CV] END .....clf__C=0.5, clf__penalty=l2, clf__solver=lbfgs; total time=  15.2s
[CV] END .....clf__C=0.5, clf__penalty=l2, clf__solver=lbfgs; total time=  12.6s
[CV] END .clf__C=0.5, clf__penalty=l2, clf__solver=liblinear; total time=  14.3s
[CV] END .clf__C=0.5, clf__penalty=l2, clf__sol